Preliminary Note: Recall that I cannot use any specifically markdown formatting in the notebook, since opening it in PyCharm (a necessary operation to ensure it has saved code appropriately) will permanently delete any cell with special markdown formatting.

Vision Revision:
For my final project, I propose to build a neural network trained to generate poetry in the style of John  Milton. My dataset will be a simple text file containing the entirety of Paradise Lost (about 11,000 lines of unrhyming iambic pentameter (also known as blank verse)). My hope for this project is that it can learn how to generate real words (or at least words that sound real) as well as poetry with the right number of syllables per line.

Background:
This project will use an LSTM (Long short-term memory) trained on the text of Paradise Lost. An LSTM is a specific type of recurrent neural network (RNN) which is structured in order to be able to learn longer term interdependencies between inputs. This LSTM will analyze the text of Paradise Lost, tokenized by letter, and will generate output coorespondingly. My model for this will be Chollet (https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/8.1-text-generation-with-lstm.ipynb) with some cross-referencing of another example (https://github.com/pranjal52/text_generators) which was unfortunately running too slow on the lab computers to be very useful, but did provide some structural guidance. While it might seem intuitively obvious to tokenize this sort of data at the level of words, or possibly syllables with some additional background work, almost every single useful example I could find of textual generation with an LSTM tokenizes at the level of individual characters (letters and punctuation). For the purposes of this project, I have assumed that there is a legitimate reason for this, althouh I am not sure what that reason is. My best guess is that, for datasets that would take a shorter amount of time than a week to run (or datasets of a fixed, specifically small size, such as this one), there just isn't enough word data to tokenize at a level of resolution that high. In otherwords, tokenizing at the level of individual letters rather than words effectively (at least) triples the size of your dataset for learning, and still produces somewhat interesting results. Other examples (than the above two) who tokenize by letter are listed below.
https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/
http://karpathy.github.io/2015/05/21/rnn-effectiveness/
https://medium.freecodecamp.org/applied-introduction-to-lstms-for-text-generation-380158b29fb3
https://towardsdatascience.com/generating-text-using-a-recurrent-neural-network-1c3bfee27a5e

Implementation:
My implementation is primarily based on Chollet's, linked above. This implementation extracts sequences of a given character length from the text. These sequences overlap partially, are one-hot encoded, and get stored neatly in an array. We then prepare an array of target characters that come after each sequence.
The LSTM itself uses a single, medium layer of nodes to learn these sequences and which characters typically follow which other characters. It then has a dense, softmax layer to provide us with character probabilities.
To generate text, we take a seed of a specific length at random from the text, draw a probability distribution over the next character according to the current sequence, and randomly choose according to that distribution what the next character will be. We use a "temperature" variable to control how easy or difficult it is for the randomly chosen character to deviate from the most likely character. We generate text at several different temperatures in order to see which level of variation is the most like real text.

Results:
Before we get to the results proper, there are a few oddities worth noting about the LSTM. And these oddities are results themselves of this project, even if they are more meta-observations that I have gathered through my tinkering with the LSTM than strict results. Firstly, the LSTM only uses one layer, and additional layers do not seem to improve the performance noticeably, while they do drastically increase the training time. In addition, training the model for multiple epochs comes at no noticeable benefit to offset the increased training time. It is not perfectly clear to me why either of these are the case. That being said, my guess is that the weights in the LSTM get affected by the vanishing gradient problem, and so do not get adjusted with the additional training of more epochs, and do not benefit from the depth of more layers.
As for the results proper, they are promising! The generated text is somewhat believable, at least at lower temperatures, and I was able to almost learn the line-breaks appropriate to iambic pentameter. It is supposed to have 10 syllables per line (and a particular pattern of stressed/unstressed syllables, but that is too subtle for this network, I think), and it is usually a few syllables short or long, but in the right range. In addition, it has learned some of Milton's stylistic, poetic quirks, such as saving a syllable by dropping vowels at the end of words, as in "heav'n" and "seem'd." One particular oddity of my LSTM is its affinity for beginnin words with "s" and its overuse of "the" and "and." While of course those are very common words, it uses them even more often than I would expect of real text. And Milton does not seem to begin a disproportionate amount of words with "s," so both of these quirks have stymied me to some degree. As a final observation, it is rather interesting to observe that, at higher temperatures, the text that is generated seems very much like a language, just one that you don't know. If you read the highest temperature out loud, for example, it often sounds sort of like hearing someone read The Canterbury Tales, in Middle English, untranslated. (For an example of what I mean, see: https://www.youtube.com/watch?v=GihrWuysnrc). It is for that reason that I consider this project a something of a success. Even if the poetry it generates is not very believably Miltonic, it does seem to have picked up on many of the subtleties of intra-word-structure such that it can generate text that sounds like a language but is definitely not.

Implications:
Poetry is weird. A conglomeration of semi-coherent, half-connected words and phrases whose structure and interplay somehow amount to far more than their mere parts -- it does not seem like the sort of thing that a NN or AI (at least currently) could convincingly fake. It's too human, too subjective, and toes the line between nonsense and beauty far too closely. But the fact that it is subjective and often so open to interpretation is exactly what makes the potential for NN-generated poetry so great. If someone doesn't know that a poem has been generated by an un-thinking AI (and no AI that I am aware of can currently be described as thinking, in a human sense), they will impute their own interpretations and understandings into the poem and attribute them to the author. In other words, if a neural network can generate convingingly human poetry, it is due in large part to how nebulous and varying poetry is in the first place. But that fact is simultaneously why it would be difficult to train a neural network to generate convincing poetry. The variation and inconsistency that makes poetry such a uniquely human affair also makes it difficult to train a neural network on. This is one of the reasons why I have trouble believing any AI will ever be up to or exceed the capacity of a human beings in their own domains (obviously computers can easily surpass human beings at, say, raw computational power and things of that sort). There is something about us that makes us and our cultural activities distinct from, and greater than, mere mathematics.

In [77]:
import keras
import numpy as np

text = open("projectData.txt").read().lower()

# I tested the generator on other text than Milton, too. alternativeData.txt currently 
# contains several of Charles Dickin's novels, amounting to a dataset about 10x the size of Milton's.
#text = open("alternativeData.txt").read().lower()

print('Corpus length:', len(text))

Corpus length: 454312


In [108]:
# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
print(chars)
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 151418
Unique characters: 38
['\n', ' ', '!', "'", '(', ')', ',', '-', '.', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Vectorization...


In [109]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))
print("Done")

Done


In [110]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
print("Done")

Done


In [111]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

print("Done")

Done


In [112]:
import random
import sys

print("epoch")
# Fit the model for 1 epochs on the available training data
model.fit(x, y, batch_size=32, epochs=1)

epoch
Epoch 1/1
151418/151418 [==============================] - 94s 622us/step - loss: 1.9888


In [113]:
# Select a text seed at random
start_index = random.randint(0, len(text) - maxlen - 1)
generated_text = text[start_index: start_index + maxlen]
print('--- Generating with seed: "' + generated_text + '"')

for temperature in [0.2, 0.5, 1.0, 1.2]:
    print('------ temperature:', temperature)
    sys.stdout.write(generated_text)

    # We generate 400 characters
    for i in range(400):
        sampled = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(generated_text):
            sampled[0, t, char_indices[char]] = 1.

        preds = model.predict(sampled, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = chars[next_index]

        generated_text += next_char
        generated_text = generated_text[1:]

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

--- Generating with seed: "de
abandond at the terror of thy power
or potent tongue; foo"
------ temperature: 0.2
de
abandond at the terror of thy power
or potent tongue; food and the self the first heav'n
the see when the self the first his seems
and shall the seat the confould the seat and her shall shall see
he with his where the self the seems the seems
the seare and shall the san and power the flame
the seake the sen shall the seems
of the heav'n and heav'n the seems and shall shall shall stand
and the more the sernation, the sernal heav'n the first the flame
of 
------ temperature: 0.5
ore the sernation, the sernal heav'n the first the flame
of the world and the argh for at man her shall
all the light arming, and heav'n
of till the felse not when man when first lage hill
death his warrie and so return'd,
the man his fouchion, and heav'n and the more his prone
with to while his his for seakes endure the fhame the sere,
not our as the suporrusion rotures of lave.
have the restempers